In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.externals import joblib
import time
from incremental_trees.trees import StreamingRFC

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
city = 'Curitiba'

data_path = '../../data/output/' + city + '/normalized_data_X_FE.csv'
y_path = '../../data/output/' + city + '/y_FE.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(6211570, 116)
(6211570, 1)


,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,stopPointId,...,alertSubtype_code,alertType_code,alertRoadType_code,jamBlockType_code,busCodeSB_code,alertSubtypeSB_code,alertTypeSB_code,alertRoadTypeSB_code,jamBlockTypeSB_code,problemSB_code
0,0.0,0.899945,0.916733,0.788385,0.014696,0.000000,0.348779,0.517589,0.000000,0.550342,...,0.814815,0.666667,0.25,0.25,0.015395,0.814815,0.666667,0.25,0.333333,0.000000
1,0.0,0.899947,0.921318,0.772879,0.022706,0.000000,0.350518,0.513454,0.000000,0.550100,...,0.814815,0.666667,0.25,0.25,0.015395,0.814815,0.666667,0.25,0.333333,0.000000
2,0.0,0.899949,0.927984,0.771251,0.028732,0.000000,0.353047,0.513020,0.000000,0.653710,...,0.814815,0.666667,0.25,0.25,0.015395,0.814815,0.666667,0.25,0.333333,0.000000
3,0.0,0.899954,0.938787,0.776257,0.037058,0.012792,0.357100,0.514376,0.000048,0.653771,...,0.814815,0.666667,0.25,0.25,0.015395,0.814815,0.666667,0.25,0.333333,0.333333
4,0.0,0.899958,0.948479,0.772553,0.044445,0.000000,0.360821,0.513367,0.000000,0.653831,...,0.814815,0.666667,0.25,0.25,0.015395,0.814815,0.666667,0.25,0.333333,0.000000


In [3]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)
train_X_1, train_X_2, train_Y_1, train_Y_2 = train_test_split(train_X, train_Y, test_size=0.50, random_state=7) 

In [4]:
# proportions to split data

#citya
#perc20 = 286526
#perc40 = 573053

#curitiba
perc20 = 1242314
perc40 = 2484628

In [5]:
#20%
first_20perc_X = train_X_2[0:perc20]
second_20perc_X = train_X_2[perc20:perc40]
#third_20perc_X = train_X_2[573054:859580]
first_20perc_Y = train_Y_2[0:perc20]
second_20perc_Y = train_Y_2[perc20:perc40]
#third_20perc_Y = train_Y_2[573054:859580]

print(first_20perc_X.shape, first_20perc_Y.shape)
print(second_20perc_X.shape, second_20perc_Y.shape)
#print(third_10perc_X.shape, third_10perc_Y.shape)

(1242314, 116) (1242314, 1)
(1242314, 116) (1242314, 1)


In [4]:
#1%
first_1perc_X = train_X_2[0:350]
second_1perc_X = train_X_2[351:701]
third_1perc_X = train_X_2[702:1052]
first_1perc_Y = train_Y_2[0:350]
second_1perc_Y = train_Y_2[351:701]
third_1perc_Y = train_Y_2[702:1052]

print(first_1perc_X.shape, first_1perc_Y.shape)
print(second_1perc_X.shape, second_1perc_Y.shape)
print(third_1perc_X.shape, third_1perc_Y.shape)

In [10]:
#10%
first_10perc_X = train_X_2[0:143263]
second_10perc_X = train_X_2[143264:286527]
third_10perc_X = train_X_2[286528:429791]
first_10perc_Y = train_Y_2[0:143263]
second_10perc_Y = train_Y_2[143264:286527]
third_10perc_Y = train_Y_2[286528:429791]

print(first_10perc_X.shape, first_10perc_Y.shape)
print(second_10perc_X.shape, second_10perc_Y.shape)
print(third_10perc_X.shape, third_10perc_Y.shape)

In [6]:
del data
del y

### Majoriting Voting

In [10]:
def get_quality(pred):
    accuracy = accuracy_score(test_Y, pred)
    precision = precision_score(test_Y, pred)
    recall = recall_score(test_Y, pred)
    f_measure = f1_score(test_Y, pred)
    
    print("Accuracy: " + str(accuracy))
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F-measure: " + str(f_measure))

#### CatBoost

In [17]:
# Curitiba
best_learning_rate = 0.1
best_iterations = 10000
best_l2_leaf_reg = 5

In [7]:
# City A
best_learning_rate = 0.01
best_iterations = 100000
best_l2_leaf_reg = 1

In [ ]:
# parameters with manual search
best_learning_rate = 0.9
best_iterations = 10000
best_l2_leaf_reg = 5

In [13]:
#trainig catboost with 50% of data

start = time.time()
model_1 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=best_iterations,
    learning_rate=best_learning_rate,
    random_seed=42,
    depth=8,
    l2_leaf_reg=best_l2_leaf_reg,
    logging_level='Silent'
)
model_1.fit(train_X_1, train_Y_1)
print("Execution time: " + str((time.time() - start)/60) + " min")

model_1.save_model('models/cat1_50perc_grid_' + city + '.model')

Execution time: 81.33576728900273 min


In [18]:
#adding 20% of increments in the last model (with 50%)
model_2_v2 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=best_iterations,
    learning_rate=best_learning_rate,
    random_seed=42,
    depth=8,
    l2_leaf_reg=best_l2_leaf_reg,
    logging_level='Silent'
)

cat1_time = time.time()
model_2_v2.fit(first_20perc_X, first_20perc_Y, init_model='models/cat1_50perc_grid_' + city + '.model')
print("Execution time: " + str((time.time() - cat1_time)/60) + " min")

model_2_v2.save_model('models/cat2_20perc_grid_' + city + '.model')

Execution time: 38.7939369002978 min


In [26]:
#adding 20% of increments in the last model (with 70%)
model_2_v3 = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=best_iterations,
    learning_rate=best_learning_rate,
    random_seed=42,
    depth=8,
    l2_leaf_reg=best_l2_leaf_reg,
    logging_level='Silent'
)
cat2_time = time.time()
model_2_v3.fit(second_20perc_X, second_20perc_Y, init_model='models/cat2_20perc_grid_' + city + '.model')
print("Execution time: " + str((time.time() - cat2_time)/60) + " min")

model_2_v3.save_model('models/cat3_40perc_grid_' + city + '.model')

Execution time: 38.961919685204826 min


In [ ]:
#model_1 = CatBoostClassifier().load_model('models/cat1_50perc_grid.model')

Curitiba

In [14]:
# 50% all data - grid
cat_pred = model_1.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9694626318306
Precision: 0.8128694509909979
Recall: 0.5983164839539877
F-measure: 0.689282935419141


In [19]:
# 50%+20% all data - grid
cat_pred = model_2_v2.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9704374256427925
Precision: 0.8129062296303194
Recall: 0.6206401342262794
F-measure: 0.7038798942140232


In [27]:
# 50%+40% all data - grid
cat_pred = model_2_v3.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9716778527811809
Precision: 0.8139763427795447
Recall: 0.6477413300345519
F-measure: 0.7214062314422581


City A

In [19]:
# 50% all data - grid
cat_pred = model_1.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9294621449287501
Precision: 0.8534642032332563
Recall: 0.5747725328563652
F-measure: 0.6869278312189229


In [24]:
# 50%+20% all data - grid
cat_pred = model_2_v2.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9304952063854366
Precision: 0.8333452414975707
Recall: 0.6046867304352335
F-measure: 0.7008367258033019


In [10]:
# 50%+40% all data - grid
cat_pred = model_2_v3.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9349066580112869
Precision: 0.8331215728233249
Recall: 0.6459029991964124
F-measure: 0.7276629918960357


In [24]:
# Bus Bunching 50%-40% all data - new rf
cat_pred = model_2_v3.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9147340390259906
Precision: 0.6991272522522523
Recall: 0.6437255359411048
F-measure: 0.6702835472421286


In [14]:
# Bus Bunching 50%-20% all data - new rf
cat_pred = model_2_v2.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9200354591364862
Precision: 0.731108553893364
Recall: 0.6422998159525105
F-measure: 0.683832864160733


In [34]:
# Bus Bunching 50%-3% all data
cat_pred = model_2_v4.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.8904466245763925
Precision: 0.6668678894822382
Recall: 0.3722684501127615
F-measure: 0.47780809156241677


In [26]:
# Bus Bunching 50%-2% all data
cat_pred = model_2_v3.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.8981317641967423
Precision: 0.7285429141716567
Recall: 0.38792544780568733
F-measure: 0.5062755844243716


In [18]:
# Bus Bunching 50%-1% all data
cat_pred = model_2_v2.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9057122016424282
Precision: 0.7792916847852346
Recall: 0.4180988672006636
F-measure: 0.5442183756790497


In [9]:
# Bus Bunching 50% all data
cat_pred = model_1.predict(test_X)
get_quality(cat_pred)

Accuracy: 0.9291096476073808
Precision: 0.7820152549177037
Recall: 0.6564533271120098
F-measure: 0.7137542277339346


In [26]:
cat_pred = model_2_v4.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9379695213565142
Precision: 0.7379110251450677
Recall: 0.5614422369389257
F-measure: 0.63769327204346


In [18]:
cat_pred = model_2_v3.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9425484724905201
Precision: 0.7477718360071302
Recall: 0.6173657100809419
F-measure: 0.6763401854091092


In [11]:
cat_pred = model_2_v2.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9479859769621521
Precision: 0.8257731958762886
Recall: 0.5894039735099338
F-measure: 0.6878488621726063


In [6]:
cat_pred = model_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9291131376798696
Precision: 0.7617805549185966
Recall: 0.6889338206703476
F-measure: 0.7235282107125842


In [34]:
cat_pred = model_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, cat_pred)
precision = precision_score(test_Y.busBunching.values, cat_pred)
recall = recall_score(test_Y.busBunching.values, cat_pred)
f_measure = f1_score(test_Y.busBunching.values, cat_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9567146025613508
Precision: 0.851024208566108
Recall: 0.6725533480500367
F-measure: 0.7513357994245787


#### XGBoost

In [11]:
# City A
best_estimator = 25
best_depth = 5
best_learning_rate = 0.01

In [8]:
# Curitiba
best_estimator = 150
best_depth = 50
best_learning_rate = 0.1

In [ ]:
# parameters with manual search
best_estimator = 120
best_depth = 50
best_learning_rate = 0.1

In [9]:
#trainig xgboost with 50% of data

start = time.time()
xgb_1 = XGBClassifier(
     learning_rate=best_learning_rate,
     n_estimators=best_estimator, 
     max_depth=best_depth,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)
xgb_1.fit(train_X_1, train_Y_1)
print("Execution time: " + str((time.time() - start)/60) + " min")

xgb_1.save_model('models/xgb1_50perc_grid_' + city + '.model')

/home/veruska/.local/lib/python3.6/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, wa

[20:15:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Execution time: 59.28151110013326 min


In [20]:
#adding 20% of increments in the last model (with 50%)

xgb_2_v2 = XGBClassifier(
     learning_rate=best_learning_rate,
     n_estimators=best_estimator, 
     max_depth=best_depth,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)

xgb1_time = time.time()
xgb_2_v2.fit(first_20perc_X, first_20perc_Y, xgb_model='models/xgb1_50perc_grid_' + city + '.model')
print("Execution time: " + str((time.time() - xgb1_time)/60) + " min")

xgb_2_v2.save_model('models/xgb2_20perc_grid_' + city + '.model')

/home/veruska/.local/lib/python3.6/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, wa

[20:30:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Execution time: 19.53485087156296 min


In [28]:
#adding 20% of increments in the last model (with 70%)

xgb_2_v3 = XGBClassifier(
     learning_rate=best_learning_rate,
     n_estimators=best_estimator, 
     max_depth=best_depth,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=-1,
     scale_pos_weight=1,
     seed=23)

xgb2_time = time.time()
xgb_2_v3.fit(second_20perc_X, second_20perc_Y, xgb_model='models/xgb2_20perc_grid_' + city + '.model')
print("Execution time (min): " + str((time.time() - xgb2_time)/60))

xgb_2_v3.save_model('models/xgb3_40perc_grid_' + city + '.model')

/home/veruska/.local/lib/python3.6/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, wa

[21:39:28] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Execution time (min): 19.5987198193868


Curitiba

In [11]:
# 50% all data - grid
xgb_pred = xgb_1.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9758386366087801
Precision: 0.8582931901807775
Recall: 0.6865588875143966
F-measure: 0.7628805713112035


In [21]:
# 50%+20% all data - grid
xgb_pred = xgb_2_v2.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9765091595200569
Precision: 0.8529300761682563
Recall: 0.7069487693554579
F-measure: 0.7731085903546077


In [29]:
# 50%+40% all data - grid
xgb_pred = xgb_2_v3.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9769752252651102
Precision: 0.8492157814566211
Recall: 0.7213667192765431
F-measure: 0.7800876451141693


City A

In [18]:
# 50% all data - grid
xgb_pred = xgb_1.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.8673283844105442
Precision: 0.7812187812187812
Recall: 0.020271146019649015
F-measure: 0.039516903330132906


In [26]:
# 50%+20% all data - grid
xgb_pred = xgb_2_v2.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.8686755523912232
Precision: 0.8860862489829129
Recall: 0.028229255774165955
F-measure: 0.05471536954228006


In [13]:
# 50%+40% all data - grid
xgb_pred = xgb_2_v3.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.8684591678969172
Precision: 0.9080036798528058
Recall: 0.02558519324986391
F-measure: 0.049768051633723276


In [26]:
# Bus Bunching 50%-40% all data - new rf
xgb_pred = xgb_2_v3.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9336572120602945
Precision: 0.8431056249123299
Recall: 0.6232210902869586
F-measure: 0.7166768515344373


In [16]:
# Bus Bunching 50%-20% all data - new rf
xgb_pred = xgb_2_v2.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9327253627057834
Precision: 0.8568048212371058
Recall: 0.6007206366487804
F-measure: 0.7062660002438133


In [36]:
# Bus Bunching 50%-3% all data
xgb_pred = xgb_2_v4.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.910507561242047
Precision: 0.823423513527029
Recall: 0.42683464240350466
F-measure: 0.562228975313279


In [28]:
# Bus Bunching 50%-2% all data
xgb_pred = xgb_2_v3.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9150830462748711
Precision: 0.8164652567975831
Recall: 0.47637193146175183
F-measure: 0.6016861750020464


In [20]:
# Bus Bunching 50%-1% all data
xgb_pred = xgb_2_v2.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9209568382735309
Precision: 0.8319509857041637
Recall: 0.517432667133266
F-measure: 0.6380373981141122


In [11]:
# Bus Bunching 50% all data
xgb_pred = xgb_1.predict(test_X)
get_quality(xgb_pred)

Accuracy: 0.9325299186464103
Precision: 0.879271609317725
Recall: 0.5782720273738238
F-measure: 0.6976918746481516


In [28]:
xgb_pred = xgb_2_v4.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9333905702225084
Precision: 0.6804384485666105
Recall: 0.5938189845474614
F-measure: 0.6341846758349705


In [20]:
xgb_pred = xgb_2_v3.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.943478571939615
Precision: 0.7847847847847848
Recall: 0.5768947755702722
F-measure: 0.6649703138252756


In [14]:
xgb_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9483437075194964
Precision: 0.8503850385038504
Recall: 0.5688005886681383
F-measure: 0.681657848324515


In [9]:
xgb_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [36]:
xgb_pred = xgb_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, xgb_pred)
precision = precision_score(test_Y.busBunching.values, xgb_pred)
recall = recall_score(test_Y.busBunching.values, xgb_pred)
f_measure = f1_score(test_Y.busBunching.values, xgb_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9567861486728196
Precision: 0.9180509413067552
Recall: 0.6100073583517293
F-measure: 0.7329796640141468


#### Random Forest

In [12]:
#trainig rf with 50% of data

start = time.time()
random.seed(42)

# Curitiba
rf_1 = RandomForestClassifier(n_estimators=25, min_samples_split=5,
                           max_features=0.5, n_jobs=-1, warm_start=True)

# City A
#rf_1 = RandomForestClassifier(n_estimators=25, min_samples_split=15,
#                           max_features=0.9, n_jobs=-1, warm_start=True)

# parameters with manual search
#rf_1 = RandomForestClassifier(n_estimators=100, min_samples_split=5,
#                           max_features=0.8, n_jobs=-1, warm_start=True)

rf_1.fit(train_X_1, train_Y_1)

print("Execution time: " + str((time.time() - start)/60) + " min")

/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Execution time: 6.281097590923309 min


In [22]:
#adding 20% of increments in the last model (with 50%)

random.seed(42)
rf1_time = time.time()
rf_1.fit(first_20perc_X, first_20perc_Y)
print("Execution time (min): " + str(time.time() - rf1_time))

Execution time (min): 0.3027198314666748


/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/home/veruska/.local/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [30]:
#adding 20% of increments in the last model (with 70%)

random.seed(42)
rf2_time = time.time()
rf_1.fit(second_20perc_X, second_20perc_Y)
print("Execution time (min): " + str(time.time() - rf2_time))

/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Execution time (min): 3.1898694038391113


/home/veruska/.local/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


City A

In [16]:
# 50% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9264118215735341
Precision: 0.8604219896150993
Recall: 0.5412292298519843
F-measure: 0.6644813264802762


In [30]:
# 50%+20% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9267014975901049
Precision: 0.8636758680627405
Recall: 0.5409700080358762
F-measure: 0.6652534268409308


In [17]:
# 50%+40% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9268061997647691
Precision: 0.8647192873420344
Recall: 0.5409959302174872
F-measure: 0.6655823446868223


Curitiba

In [13]:
# 50% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9735388959635004
Precision: 0.8431419253178937
Recall: 0.6543104551465256
F-measure: 0.7368201942245031


In [23]:
# 50%+20% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.973540505862447
Precision: 0.8431853996408546
Recall: 0.6542962362610019
F-measure: 0.7368277783560042


In [31]:
# 50%+40% all data - grid
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.973538091014027
Precision: 0.8431264772165118
Recall: 0.6543104551465256
F-measure: 0.7368142953213564


In [17]:
rf_1 = StreamingRFC(n_estimators=100, min_samples_split=5, max_features=0.8, n_jobs=-1)

rf1_time = time.time()
rf_1.partial_fit(train_X_1, train_Y_1, classes=np.unique(train_Y_1))
print("Execution time (sec): " + str(time.time() - rf1_time))

/home/veruska/.local/lib/python3.6/site-packages/incremental_trees/trees.py:201: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  super().fit(*args)


Execution time (sec): 18.953135013580322


In [18]:
rf1_time = time.time()
rf_1.partial_fit(first_20perc_X, first_20perc_Y)
print("Execution time (sec): " + str(time.time() - rf1_time))

/home/veruska/.local/lib/python3.6/site-packages/incremental_trees/trees.py:201: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  super().fit(*args)


Execution time (sec): 10.187557697296143


In [27]:
rf2_time = time.time()
rf_1.partial_fit(second_20perc_X, second_20perc_Y)
print("Execution time (sec): " + str(time.time() - rf2_time))

/home/veruska/.local/lib/python3.6/site-packages/incremental_trees/trees.py:201: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  super().fit(*args)


Execution time (sec): 9.983682632446289


In [21]:
rf3_time = time.time()
rf_1.partial_fit(third_10perc_X, third_10perc_Y)
print("Execution time (sec): " + str(time.time() - rf3_time))

/home/veruska/.local/lib/python3.6/site-packages/incremental_trees/trees.py:201: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  super().fit(*args)


Execution time (sec): 4.9929211139678955


In [28]:
# Bus Bunching 50%-40% all data - new rf
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.896665933751444
Precision: 0.6620532649152604
Recall: 0.4749202892915468
F-measure: 0.5530867924528302


In [19]:
# Bus Bunching 50%-20% all data - new rf
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.8924952971273213
Precision: 0.6951109326372854
Recall: 0.35897037094641887
F-measure: 0.47344399049556407


In [22]:
# Bus Bunching 50%-30% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.8901185577624448
Precision: 0.696058378019791
Recall: 0.32638618866163777
F-measure: 0.4443934634525112


In [20]:
# Bus Bunching 50%-20% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.886021212660587
Precision: 0.6130054603077628
Recall: 0.4161547035798533
F-measure: 0.4957539449711269


In [18]:
# Bus Bunching 50%-10% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.8871240755670495
Precision: 0.6705882352941176
Recall: 0.3176763356404075
F-measure: 0.43111939773446845


In [15]:
# Bus Bunching 50% all data - partialFit
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.8791143592052407
Precision: 0.5508544581547834
Recall: 0.5531534333929543
F-measure: 0.5520015520920909


In [38]:
# Bus Bunching 50%-3% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [30]:
# Bus Bunching 50%-2% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [22]:
# Bus Bunching 50%-1% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [13]:
# Bus Bunching 50% all data
rf_pred = rf_1.predict(test_X)
get_quality(rf_pred)

Accuracy: 0.9345402003999623
Precision: 0.8786246418338108
Recall: 0.5961583326852788
F-measure: 0.7103409933283915


In [30]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [22]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [8]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9559991414466624
Precision: 0.903470715835141
Recall: 0.6129506990434143
F-measure: 0.7303814116615519


In [12]:
rf_pred = xgb_2_v2.predict(test_X)

# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [38]:
rf_pred = rf_1.predict(test_X)

# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, rf_pred)
precision = precision_score(test_Y.busBunching.values, rf_pred)
recall = recall_score(test_Y.busBunching.values, rf_pred)
f_measure = f1_score(test_Y.busBunching.values, rf_pred)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.957072333118695
Precision: 0.9129488574537541
Recall: 0.6173657100809419
F-measure: 0.736611062335382


### Ensemble Evaluation

In [32]:
final_predictions = []

for i in range(0, len(rf_pred)):
    temp_preds = []
    temp_preds.append(rf_pred[i])
    temp_preds.append(cat_pred[i])
    temp_preds.append(xgb_pred[i])
    
    final_predictions.append(
        max(temp_preds,key=temp_preds.count)
    )

City A

In [22]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_' + city + '.csv', index=False)

get_quality(final_predictions) #50% - grid

Accuracy: 0.9233021669860083
Precision: 0.9018639554587267
Recall: 0.4828783990460637
F-measure: 0.6289843327930849


In [32]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_20_' + city + '.csv', index=False)

get_quality(final_predictions) #50%+20% - grid

Accuracy: 0.9227088546629113
Precision: 0.9221953851688165
Recall: 0.4651735490058843
F-measure: 0.6184092632159348


In [19]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_40_' + city + '.csv', index=False)

get_quality(final_predictions) #50%+40% - grid

Accuracy: 0.9243491887326499
Precision: 0.9268576046875789
Recall: 0.4756461103766493
F-measure: 0.6286702984205297


Curitiba

In [16]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_' + city + '.csv', index=False)

get_quality(final_predictions) #50% - grid

Accuracy: 0.9746207480556446
Precision: 0.8589933382679497
Recall: 0.6600406660125979
F-measure: 0.7464882728011001


In [25]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_20_' + city + '.csv', index=False)

get_quality(final_predictions) #50%+20% - grid

Accuracy: 0.9755802478278438
Precision: 0.8622988838962169
Recall: 0.6767051998464361
F-measure: 0.7583113582587774


In [33]:
preds_pd = pd.DataFrame(final_predictions, columns=['pred'])
preds_pd.to_csv('../Ensemble/outputs/q5/ens_pred_q5_50_40_' + city + '.csv', index=False)

get_quality(final_predictions) #50%+40% - grid

Accuracy: 0.97629423801068
Precision: 0.8641781737193764
Recall: 0.689644385673051
F-measure: 0.7671089882486912


In [30]:
# Bus Bunching 50%-40% all data - new rf
get_quality(final_predictions)

Accuracy: 0.9313467840727052
Precision: 0.8405259365994237
Recall: 0.6048422635248982
F-measure: 0.703468652486546


In [21]:
# Bus Bunching 50%-20% all data - new rf
get_quality(final_predictions)

Accuracy: 0.9298530330474964
Precision: 0.8599142968445657
Recall: 0.5722062368768955
F-measure: 0.6871604899839682


In [40]:
# Bus Bunching 50%-3% all data
get_quality(final_predictions)

Accuracy: 0.9190966296369976
Precision: 0.8795857988165681
Recall: 0.46239987557352824
F-measure: 0.6061471022987919


In [32]:
# Bus Bunching 50%-2% all data
get_quality(final_predictions)

Accuracy: 0.9225169006760271
Precision: 0.8739495798319328
Recall: 0.4960468673043523
F-measure: 0.632877482512857


In [24]:
# Bus Bunching 50%-1% all data
get_quality(final_predictions)

Accuracy: 0.9257417276556834
Precision: 0.8769720212673233
Recall: 0.5216320605542163
F-measure: 0.6541618581668646


In [15]:
# Bus Bunching 50% all data
get_quality(final_predictions)

Accuracy: 0.9361037528749472
Precision: 0.8803527866391443
Recall: 0.608056614044638
F-measure: 0.7192971696666768


In [32]:
# Bus Bunching 50%-3%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9497746297488732
Precision: 0.841831425598335
Recall: 0.5952906548933039
F-measure: 0.6974137931034482


In [24]:
# Bus Bunching 50%-2%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9530657508764399
Precision: 0.8627450980392157
Recall: 0.6151582045621781
F-measure: 0.7182130584192441


In [16]:
# Bus Bunching 50%-1%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9522787436502826
Precision: 0.8922902494331065
Recall: 0.579102281089036
F-measure: 0.7023650156180277


In [14]:
# Bus Bunching 50%-50%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9390074931856335
Precision: 0.8725590592888167
Recall: 0.6405371076029759
F-measure: 0.7387586701745994


In [40]:
# Bus Bunching 50%
accuracy = accuracy_score(test_Y.busBunching.values, final_predictions)
precision = precision_score(test_Y.busBunching.values, final_predictions)
recall = recall_score(test_Y.busBunching.values, final_predictions)
f_measure = f1_score(test_Y.busBunching.values, final_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9583601631251342
Precision: 0.9155080213903743
Recall: 0.6298749080206034
F-measure: 0.7462946817785527
